In [1]:
%matplotlib inline

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

import pandas as pd
from Utils import *

from algorithms import *
from new_utils import *

# Comparison of algorithms for ordinal embedding

In [61]:
#Create data|
dimensions= 3
number_of_points= 30

X = np.random.random((number_of_points, dimensions))
X = center_data(X)

In [67]:
n,d = X.shape
pulls = 20*int(number_of_points*dimensions*np.log(number_of_points))
split_percentage = 0.8

# NOISE RUINS EVERYTHING, talk to blake about how to measure acc in this case
# winner, loser, head
tot_triplets, error = getTriplets(X, int(pulls*(2-split_percentage)), noise=False) 


triplets = tot_triplets[:int(len(tot_triplets)*split_percentage)]
test_triplets = tot_triplets[int(len(tot_triplets)*split_percentage):]

print('Dimensions:{} \nNumber of points:{} \nPulls:{}'.format(dimensions,number_of_points, len(tot_triplets)))
print()
print('TRAIN: ', len(triplets))
print('TEST: ', len(test_triplets))


Dimensions:3 
Number of points:30 
Pulls:7344

TRAIN:  5875
TEST:  1469


In [69]:
from sklearn.metrics import accuracy_score

def predictX(X_true, X_predict, test_triplets):
    
    true = []
    pred = []
    for q in test_triplets:
        
        true_label = np.sign(scoreX(X_true, q, 1))
        pred_label = np.sign(scoreX(X_predict, q, 1))
        
        true.append(true_label)
        pred.append(pred_label)
    
    acc = accuracy_score(true, pred)
    print("Correctly predicted {0:.2f} % of the unseen triplets".format(acc*100))    
    
    return acc

def predictM(M_true, M_predict, test_triplets):
    
    true = []
    pred = []
    for q in test_triplets:
        
        true_label = np.sign(scoreM(M_true, q, 1))
        pred_label = np.sign(scoreM(M_predict, q, 1))
        
        true.append(true_label)
        pred.append(pred_label)
    
    acc = accuracy_score(true, pred)
    print("Correctly predicted {0:.2f} % of the unseen triplets".format(acc*100))    
    
    return acc

# Algorithms

In [64]:
# from projected_gradient import *

# M, loss, proj_grad_loss_arr = computeEmbedding(M0,
#                 n,
#                  d,
#                  triplets,
#                  num_random_restarts=0,œ
#                  max_iter_GD=500,
#                  max_norm=1,
#                  epsilon=0.0001,
#                  accuracy=accuracy, 
#                  verbose=False)

# from crowd_kernel import *
# X, emp_loss_train, projection_free_loss_arr = computeEmbedding(n,d,triplets,alpha=1,
#                                     num_random_restarts=0,
#                                     epsilon=0.0001,
#                                     accuracy = accuracy,
#                                     max_iters=100,
#                                     verbose=False)


In [53]:
epsilons = np.linspace(0.005, 0.1,5)
epsilons

array([ 0.005  ,  0.02875,  0.0525 ,  0.07625,  0.1    ])

In [72]:
# Non Convex 
exp = 0
X0 = np.random.random((n,d))
stats_non_convex_single_exp_full_gd = triplet_algorithms(ste_loss, 
                           triplets,
                           X0,                       
                           d,
                           'full_grad', 
                           50,
                           iters=500,
                           epsilon = epsilons[exp],
                           proj=None,
                           debug=False
                          )

X_hat = stats_non_convex_single_exp['embedding']

Done
Step size was too big, halving it. Iteration #:  4
Step size was too big, halving it. Iteration #:  4
No progress
Exiting


In [55]:
predictX(X, X_hat, test_triplets)

Correctly predicted 96.24 % of the unseen triplets


0.96241830065359479

In [59]:
# CONVEX STE FULL GRAD Single experiment
M0 = X0 @ X0.T
stats_convex_single_exp = triplet_algorithms(ste_loss_convex, 
                       triplets,
                       M0,                       
                       d,                            
                       'full_grad', 
                        300,
                       iters=5000,
                       epsilon =epsilons[exp],
                       proj=projected_psd,
                       debug= True)
    
M_hat = stats_convex_single_exp['embedding']

Done
EPOCH: 1 LOG ERROR 0.7339942611433395 Emp error 0.29616013071895425
EPOCH: 2 LOG ERROR 0.48114973397651545 Emp error 0.22038398692810457
EPOCH: 3 LOG ERROR 0.4437245246492277 Emp error 0.2079248366013072
EPOCH: 4 LOG ERROR 0.47498822205921193 Emp error 0.2042483660130719
EPOCH: 5 LOG ERROR 0.37258010059274166 Emp error 0.16891339869281047
EPOCH: 6 LOG ERROR 0.32261055421846946 Emp error 0.14501633986928106
EPOCH: 7 LOG ERROR 0.2561700297888174 Emp error 0.11254084967320262
EPOCH: 8 LOG ERROR 0.20807870074058538 Emp error 0.0900735294117647
EPOCH: 9 LOG ERROR 0.17210887348844273 Emp error 0.05984477124183007
EPOCH: 10 LOG ERROR 0.14938635241649603 Emp error 0.041666666666666664
EPOCH: 11 LOG ERROR 0.13574891069992098 Emp error 0.031862745098039214
EPOCH: 12 LOG ERROR 0.12787507154518132 Emp error 0.02185457516339869
EPOCH: 13 LOG ERROR 0.1233936283918938 Emp error 0.020016339869281044
EPOCH: 14 LOG ERROR 0.12049326611570624 Emp error 0.01838235294117647
EPOCH: 15 LOG ERROR 0.118272

In [60]:
predictM(X @ X.T, M_hat, test_triplets)

Correctly predicted 96.90 % of the unseen triplets


0.96895424836601307

In [ ]:
# FUll Experimental setup
stats_non_convex = []
for epsilon in epsilons:
    print('Epsilon', epsilon)
    X0 = np.random.random((n,d))
    stats1= triplet_algorithms(ste_loss, 
                       triplets,
                       X0,                       
                       d,
                       'full_grad', 
                       20,
                       iters=1000,
                       epsilon = epsilon,
                       proj=None,
                       debug=False)
    
    stats_non_convex.append(stats1)
    print()
    


In [ ]:
# Experiment number i.e which level of accuracy 
df = pd.DataFrame([
#         stats_convex[3]['emp'],
                   stats_non_convex[exp]['emp'],
#                    stats3['emp'],
#                     list(np.array(stats4['emp'])[epoch_inds])
                  ]).T

df.columns = [
#     'Proj Grad: Full GD (constant stepsize, shrink each epoch)',
              'Non convex: Full GD (constant stepsize, shrink each epoch)', 
#               'Proj Grad: SGD (constant stepsize)',
#               'Non Convex: SGD (constant stepsize, shrink each epoch)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Epochs', fontsize=22)
ax.set_title('Epsilon= {}'.format(epsilons[exp]), fontsize=22)
ax.legend(fontsize=18);

df = []
for i in stats_non_convex:
    df.append(sum(i['time_per_iter']))
    
df = pd.DataFrame(df, columns=['Non convex FULL GD'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');        

In [ ]:
# Non Convex SGD
exp = 2
stats_non_convex_sgd = triplet_algorithms(ste_loss, 
                           triplets,
                           X0,                       
                           d,
                           'sgd', 
                           0.2,
                           iters=5000,
                           epsilon = epsilons[exp],
                           proj=None,
                           debug=True
                          )

In [ ]:
# CONVEX STE FULL GRAD Full exp set up
M0 = X0 @ X0.T
stats_convex = []
for epsilon in epsilons:
    stats2 = triplet_algorithms(ste_loss_convex, 
                       triplets,
                       M0,                       
                       d,                            
                       'full_grad', 
                        600,
                       iters=5000,
                       epsilon =epsilon,
                       proj=projected,
                       debug= False)
    
    stats_convex.append(stats2)

In [ ]:
df = []
for i in stats_convex:
    df.append(sum(i['time_per_iter']))
    
df = pd.DataFrame(df, columns=['Non convex'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');        

In [ ]:
# stats3 = triplet_algorithms(ste_loss_convex, 
#                             triplets,
#                             M0,                       
#                             d,                            
#                             'sgd', 
#                             3,
#                             iters=5000,
#                             epsilon = 0.07625,
#                             proj=projected)

In [ ]:
# stats2['emp']

In [ ]:
epoch_inds = np.linspace(0, len(stats4['emp'])-1, stats4['epoch_count'], dtype=int)

df = pd.DataFrame([stats_convex[3]['emp'],
                   stats_non_convex[3]['emp'],
#                    stats3['emp'],
                    list(np.array(stats4['emp'])[epoch_inds])
                  ]).T
df.columns = ['Proj Grad: Full GD (constant stepsize, shrink each epoch)',
              'Non convex: Full GD (constant stepsize, shrink each epoch)', 
#               'Proj Grad: SGD (constant stepsize)',
              'Non Convex: SGD (constant stepsize, shrink each epoch)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Epochs', fontsize=22)
ax.set_title('Epsilon= {}'.format(0.07625), fontsize=22)
ax.legend(fontsize=18);

In [ ]:
df = pd.DataFrame([
#                 stats2['emp'],
#                 stats1['emp'],
                   stats3['emp'],
                   stats4['emp'],                   
                  ]
                 ).T
df.columns = [
#     'Proj Grad: Full GD (constant stepsize)',
#               'Non convex full gradient descent (constant stepsize)', 
              'Proj Grad: SGD (constant stepsize)',
              'Non Convex: SGD (constant stepsize)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Iterations of full gradient descent', fontsize=22)
ax.legend(fontsize=22);

In [ ]:
sum(stats1['time_per_iter'])

In [ ]:
sum(stats2['time_per_iter'])

In [ ]:
df = []
for i,j in zip(stats_non_convex,stats_convex):
    df.append([sum(i['time_per_iter']), len(j['emp'])*stats_convex[0]['avg_time_per_iter']])
    
df = pd.DataFrame(df, columns=['Non convex', 'Convex- with projection'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');    

In [ ]:
# for i in range(5):
#     print(len(stats_convex[i]['emp']))
#     print(stats_convex[i]['time_per_iter'])
#     print()

In [ ]:
epsilons